In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
import sys

In [ ]:
# Load the datasets
euster_df = pd.read_csv('./data/euster.csv')
fond_df = pd.read_csv('./data/fond.csv')

# Clean fond data
fond_df['Date'] = pd.to_datetime(fond_df['Date'])
fond_df = fond_df.sort_values(by='Date')

# Clean euster data
euster_df['Date'] = pd.to_datetime(euster_df['DATE'])
euster_df = euster_df.drop('DATE', axis=1)
euster_df = euster_df.sort_values('Date')
euster_df["Date"] = euster_df["Date"].astype('datetime64[ns]')

In [ ]:
def calculate_non_overlapping_window_return(df, window_size=20):
    # Number of windows
    num_windows = len(df) // window_size

    # Create a new DataFrame to store the results
    results_df = pd.DataFrame(index=range(num_windows), columns=['Date', 'Annualized Return'])
    
    # Iterate over the DataFrame
    for i in range(num_windows):
        # Determine the start and end of the window
        start = i * window_size
        end = (i + 1) * window_size
        window_df = df.iloc[start:end]
        
        # Calculate the annualized return for the window
        if not window_df.empty:
            window_size_actual = len(window_df)
            annualized_return = (window_df['Price'].iloc[-1] / window_df['Price'].iloc[0])**(365/window_size_actual) - 1
            results_df.loc[i, 'Annualized Return'] = annualized_return * 100
            # middle date of the window
            results_df.loc[i, 'Date'] = window_df['Date'].iloc[window_size_actual // 2] 
    
    # Remove rows with no calculated return
    results_df = results_df.dropna()
    
    return results_df

In [ ]:
start_date_filter = pd.to_datetime("2022-08-01")
euster_filtered = euster_df[euster_df['Date'] >= start_date_filter]

In [ ]:
# Data for best r2 window size dataframe
best_r2_df = None
best_r2_window_size = None
best_r2 = 0
best_r2_median_tracking_difference =sys.maxsize

# Data for best tracking difference window size dataframe
best_td_df = None
best_td_window_size = None
best_td_r2 = 0
best_td_median_tracking_difference =sys.maxsize

# Check window sizes from 1 to 100
for i in range(1,100):
    # Calculate the annualized return for each window
    annual_returns_df = calculate_non_overlapping_window_return(fond_df[fond_df['Date'] >= start_date_filter],i)


    annual_returns_df["Date"] = annual_returns_df["Date"].astype('datetime64[ns]')

    # Merge the datasets and calculate the tracking difference
    merged_df = pd.merge(euster_filtered, annual_returns_df, left_on='Date', right_on='Date')
    merged_df['Tracking Difference'] = merged_df['VALUE'] - merged_df['Annualized Return']
    median_tracking_difference = merged_df['Tracking Difference'].median()

    r2 = r2_score(merged_df['VALUE'], merged_df['Annualized Return'])

    # Update best r2 if needed
    if r2 > best_r2:
        best_r2 = r2
        best_r2_df = annual_returns_df
        best_r2_window_size = i
        best_r2_median_tracking_difference = median_tracking_difference
        
    # Update best tracking difference if needed
    if abs(median_tracking_difference) < best_td_median_tracking_difference:
        best_td_median_tracking_difference = median_tracking_difference
        best_td_df = annual_returns_df
        best_td_window_size = i
        best_td_r2 = r2

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(best_r2_df['Date'], best_r2_df['Annualized Return'],label=f'Best r2 (window size = {best_r2_window_size}, r2={best_r2:.2f}, tracking difference={best_r2_median_tracking_difference:.2f}%)')
plt.plot(best_td_df['Date'], best_td_df['Annualized Return'],label=f'Best tracking difference (window size = {best_td_window_size}, r2={best_td_r2:.2f}, tracking difference={best_td_median_tracking_difference:.2f}%)')

# Plot EUSTER value
plt.plot(euster_filtered['Date'], euster_filtered['VALUE'], label='EUSTER Value')

plt.title('Annualized Returns and EUSTER Value over Time')
plt.xlabel('Date')

plt.ylabel('Percentage (%)')
plt.legend()
plt.grid(True)
plt.show()